In [11]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Colab Notebooks/"
!cp "/content/drive/My Drive/Colab Notebooks/PubMed-w2v.bin" "/content/PubMed-w2v.bin"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Copy of span-search (1).ipynb'   error_log.gdoc   run_model.ipynb
'Copy of span-search.ipynb'	  PubMed-w2v.bin


In [2]:
%cd /content/
!pip install -q xlrd
!git clone https://a80b7012a5c512ec3c6cb546626348b891f4f77f@github.com/bwallace/evidence-inference --branch "span-location"

/content
fatal: destination path 'evidence-inference' already exists and is not an empty directory.


In [3]:
%cd /content/evidence-inference/
%mkdir embeddings
%mv /content/PubMed-w2v.bin /content/evidence-inference/embeddings

/content/evidence-inference
mkdir: cannot create directory ‘embeddings’: File exists
mv: cannot stat '/content/PubMed-w2v.bin': No such file or directory


In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [5]:
%cd /content/evidence-inference
!git config --global user.email "lehman.e@husky.neu.edu"
!git pull origin span-location
!pip install gensim
!pip install spacy
!python -m spacy download en


/content/evidence-inference
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/bwallace/evidence-inference
 * branch            span-location -> FETCH_HEAD
   040a770..d7b18a8  span-location -> origin/span-location
Updating 040a770..d7b18a8
Fast-forward
 evidence_inference/models/model_scan.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
import os
import sys
import random

mp1 = os.path.abspath(os.path.join('.//evidence_inference/preprocess'))
mp2 = os.path.abspath(os.path.join('.//evidence_inference/models'))
mp3 = os.path.abspath(os.path.join('.//evidence_inference/models/embeddings'))

module_paths = [mp1, mp2, mp3]
for mp in module_paths:
    if mp not in sys.path:
        sys.path.append(mp)
    
import preprocessor
USE_CUDA = True

from model_scan import ScanNet, train_scan

In [0]:
train_Xy, inference_vectorizer = preprocessor.get_train_Xy(set(list(preprocessor.train_document_ids())[:100]), sections_of_interest=None, include_sentence_span_splits = True)
    
# create an internal validation set from the training data; use 90% for training and 10% for validation.
random.shuffle(train_Xy)
split_index = int(len(train_Xy) * .9)
val_Xy = train_Xy[split_index:]
train_Xy = train_Xy[:split_index]
test_Xy = preprocessor.get_Xy(set(list(preprocessor.validation_document_ids())[:25]), inference_vectorizer, sections_of_interest=None, include_sentence_span_splits = True)



In [8]:
se_scn = ScanNet(inference_vectorizer).cuda()


FileNotFoundError: ignored

In [0]:
train_scan(se_scn, inference_vectorizer, train_Xy, val_Xy, test_Xy, 50, 5, 1000)


In [0]:
torch.save(se_scn.state_dict(), "./scan_model.pth")
